# Eat Safe, Love

## Notebook Set Up

In [4]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [5]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [6]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [7]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [9]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": {"$eq": 20}}

# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print(f"Number of establishments with a Hygiene score of 20: {count}")

# Display the first document in the results using pprint
result = establishments.find_one(query)
pprint(result)

Number of establishments with a Hygiene score of 20: 0
None


In [10]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find(query))
df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [11]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "LocalAuthorityName": {"$regex": "London", "$options": "i"},  # Case-insensitive regex for "London"
    "RatingValue": {"$gte": 4}
}

# Use count_documents to display the number of documents in the result

count = establishments.count_documents(query)
print(f"Number of establishments in London with a RatingValue >= 4: {count}")

# Display the first document in the results using pprint

result = establishments.find_one(query)
pprint(result)

Number of establishments in London with a RatingValue >= 4: 0
None


In [12]:
# Convert the result to a Pandas DataFrame

results = list(establishments.find(query))
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [15]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}
}
sort = [("scores.Hygiene", 1)]
limit = 5

# Print the results

results = establishments.find(query).sort(sort).limit(limit)
for result in results:
    pprint(result)

In [16]:
# Convert result to Pandas DataFrame
results_list = list(establishments.find(query).sort(sort).limit(limit))
df = pd.DataFrame(results_list)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [17]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {"$match": {"scores.Hygiene": 0}}, 
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}}, 
    {"$sort": {"count": -1}} 
]
# Print the number of documents in the result
results = list(establishments.aggregate(pipeline))
print(f"Number of documents in the result: {len(results)}")
# Print the first 10 results
for result in results[:10]:
    pprint(result)

Number of documents in the result: 0


In [18]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []
